In [1]:
from pythonosc import dispatcher, osc_server
from pythonosc.udp_client import SimpleUDPClient
import biofeatures
import threading
import numpy as np

In [7]:
def data_handler(unused_addr, args, data1, data2, data3, data4, data5, data6):
    global ecg_data, resp_data
    global osc_client
    
    B = args[0]
    HR = args[1]
    
    # print(data5) # breathing
    # print(data6) # ECG
    
    ecg_data.append(float(data6))
    resp_data.append(float(data5))
        
    if B.is_warmed_up:
        B.set_data(resp_data[-B.buffer_length:])
    
    if HR.is_warmed_up:
        HR.set_data(ecg_data[-HR.buffer_length:])
    


In [8]:
def warmup(B, resp_data, HR, ecg_data):
    """Function to launch once the system has warmed up.
    Sets the  data initially and launches a recursive update of features.
    Parameters
    ----------
    B: object containing breathing data and features
    resp_data: respiration signal data to set
    HR: object containing ECG data and features
    ecg_data: ecg signal data to set
    """
    
    if not B.is_warmed_up:
        print("Breathing WARMUP")
        B.set_data(resp_data[-B.buffer_length:])
        B.is_warmed_up = True
        B.update_loop()
        
    if not HR.is_warmed_up:
        print("ECG WARMUP")
        HR.set_data(ecg_data[-HR.buffer_length:])
        HR.is_warmed_up = True
        HR.update_loop()
        
    breathing_factor = 1.2
    
    timer_actuation = threading.Timer(0.1, breathing_actuation, [B, breathing_factor, True] )
    timer_actuation.start()

In [3]:
def breathing_actuation(B, breathing_factor, inflate):
    global osc_client
    
    if not actuation_flag:
        return
    
    if inflate:
        print("inhale: ", B.features['avg_inhale'])
        osc_client.send_message("/actuator/inflate", 100.0)
        timer_exhale = threading.Timer(B.features['avg_inhale'] * breathing_factor, breathing_actuation, [B, breathing_factor, False])
        timer_exhale.start()

    else:
        print("exhale: ", B.features['avg_exhale'])
        osc_client.send_message("/actuator/inflate", -100.0)
        timer_inhale = threading.Timer(B.features['avg_exhale'] * breathing_factor, breathing_actuation, [B, breathing_factor, True])
        timer_inhale.start()

In [10]:
bitalino_ip = "127.0.0.1"
bitalino_port = 8000

actuator_ip = '192.168.0.100'
actuator_port = 12000

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 

actuation_flag = True

ecg_data = []
resp_data = []

B = biofeatures.breathing(data = np.ones(10), buffer_length=2000, srate=100)
HRV = biofeatures.hrv(data = np.ones(10), buffer_length=2000, srate=100)

dispatcher = dispatcher.Dispatcher()
dispatcher.map("/0/bitalino", data_handler, B, HRV)

server = osc_server.ThreadingOSCUDPServer((bitalino_ip, bitalino_port), dispatcher)
print("Serving on {}".format(server.server_address))

# Warmup
warmup_t = 10
timer_warmup = threading.Timer(warmup_t, warmup, [B,resp_data,HRV,ecg_data])
timer_warmup.start()

try: 
    server.serve_forever()
except KeyboardInterrupt:
    B.update_data_flag = False
    HRV.update_data_flag = False
    actuation_flag = False
except:
    raise

Serving on ('127.0.0.1', 8000)
Breathing WARMUP
{'breath_avg_len': 3.29, 'inhale_duration': 4.37, 'avg_inhale': 2.18, 'exhale_duration': 5.5, 'avg_exhale': 1.83, 'inhale_exhale_ratio': 0.79}
ECG WARMUP
[{'nni_mean': 564.8, 'hr_mean': 127.34, 'hr_std': 53.78, 'rmssd': 460.78, 'lf': 94360.56, 'hf': 23688.42, 'LF/HF ratio': 3.98}]
inhale:  2.18
{'breath_avg_len': 3.54, 'inhale_duration': 4.95, 'avg_inhale': 2.48, 'exhale_duration': 5.66, 'avg_exhale': 1.89, 'inhale_exhale_ratio': 0.87}
[{'nni_mean': 564.8, 'hr_mean': 127.34, 'hr_std': 53.78, 'rmssd': 460.78, 'lf': 94360.56, 'hf': 23688.42, 'LF/HF ratio': 3.98}, {'nni_mean': 651.69, 'hr_mean': 108.19, 'hr_std': 35.56, 'rmssd': 545.23, 'lf': 104650.96, 'hf': 44026.39, 'LF/HF ratio': 2.38}]
{'breath_avg_len': 3.72, 'inhale_duration': 5.73, 'avg_inhale': 2.86, 'exhale_duration': 5.44, 'avg_exhale': 1.81, 'inhale_exhale_ratio': 1.05}
[{'nni_mean': 564.8, 'hr_mean': 127.34, 'hr_std': 53.78, 'rmssd': 460.78, 'lf': 94360.56, 'hf': 23688.42, 'LF/H

{'breath_avg_len': 3.2, 'inhale_duration': 8.59, 'avg_inhale': 2.15, 'exhale_duration': 7.4, 'avg_exhale': 1.85, 'inhale_exhale_ratio': 1.16}
[{'nni_mean': 482.35, 'hr_mean': 152.31, 'hr_std': 67.03, 'rmssd': 357.86, 'lf': 44128.85, 'hf': 6646.08, 'LF/HF ratio': 6.64}, {'nni_mean': -19.65, 'hr_mean': 147.15, 'hr_std': 66.35, 'rmssd': 2679.92, 'lf': 43482.95, 'hf': 20069.41, 'LF/HF ratio': 2.17}, {'nni_mean': 483.93, 'hr_mean': 149.92, 'hr_std': 65.1, 'rmssd': 346.77, 'lf': 35412.16, 'hf': 5958.45, 'LF/HF ratio': 5.94}, {'nni_mean': -14.3, 'hr_mean': 163.36, 'hr_std': 68.96, 'rmssd': 2600.26, 'lf': 23285.74, 'hf': 12331.5, 'LF/HF ratio': 1.89}, {'nni_mean': 486.0, 'hr_mean': 147.69, 'hr_std': 63.38, 'rmssd': 334.65, 'lf': 29545.89, 'hf': 5647.37, 'LF/HF ratio': 5.23}]
{'breath_avg_len': 3.3, 'inhale_duration': 8.98, 'avg_inhale': 2.24, 'exhale_duration': 7.54, 'avg_exhale': 1.88, 'inhale_exhale_ratio': 1.19}
[{'nni_mean': -19.65, 'hr_mean': 147.15, 'hr_std': 66.35, 'rmssd': 2679.92, 'lf

{'breath_avg_len': 4.0, 'inhale_duration': 11.44, 'avg_inhale': 2.29, 'exhale_duration': 8.54, 'avg_exhale': 2.13, 'inhale_exhale_ratio': 1.34}
[{'nni_mean': -12.15, 'hr_mean': 159.61, 'hr_std': 65.98, 'rmssd': 2937.4, 'lf': 14430.57, 'hf': 14502.31, 'LF/HF ratio': 1.0}, {'nni_mean': 430.67, 'hr_mean': 159.27, 'hr_std': 61.36, 'rmssd': 190.3, 'lf': 2210.1, 'hf': 4660.52, 'LF/HF ratio': 0.47}, {'nni_mean': 405.65, 'hr_mean': 167.78, 'hr_std': 62.49, 'rmssd': 158.5, 'lf': 3512.8, 'hf': 5785.54, 'LF/HF ratio': 0.61}, {'nni_mean': 419.13, 'hr_mean': 163.61, 'hr_std': 62.23, 'rmssd': 176.85, 'lf': 6361.36, 'hf': 5986.08, 'LF/HF ratio': 1.06}, {'nni_mean': -7.42, 'hr_mean': 150.82, 'hr_std': 64.61, 'rmssd': 2996.15, 'lf': 13546.79, 'hf': 25566.01, 'LF/HF ratio': 0.53}]
{'breath_avg_len': 4.0, 'inhale_duration': 11.35, 'avg_inhale': 2.27, 'exhale_duration': 8.63, 'avg_exhale': 2.16, 'inhale_exhale_ratio': 1.32}
[{'nni_mean': 430.67, 'hr_mean': 159.27, 'hr_std': 61.36, 'rmssd': 190.3, 'lf': 22

{'breath_avg_len': 4.0, 'inhale_duration': 10.28, 'avg_inhale': 2.06, 'exhale_duration': 9.7, 'avg_exhale': 2.42, 'inhale_exhale_ratio': 1.06}
[{'nni_mean': 429.56, 'hr_mean': 156.37, 'hr_std': 58.98, 'rmssd': 152.12, 'lf': 1597.28, 'hf': 4498.61, 'LF/HF ratio': 0.36}, {'nni_mean': 421.09, 'hr_mean': 160.29, 'hr_std': 60.09, 'rmssd': 151.25, 'lf': 2328.29, 'hf': 7065.18, 'LF/HF ratio': 0.33}, {'nni_mean': 403.12, 'hr_mean': 167.23, 'hr_std': 60.95, 'rmssd': 133.74, 'lf': 4351.34, 'hf': 5202.36, 'LF/HF ratio': 0.84}, {'nni_mean': 395.71, 'hr_mean': 170.85, 'hr_std': 61.62, 'rmssd': 132.0, 'lf': 4093.28, 'hf': 5069.53, 'LF/HF ratio': 0.81}, {'nni_mean': 424.0, 'hr_mean': 158.69, 'hr_std': 58.51, 'rmssd': 149.23, 'lf': 1844.12, 'hf': 4113.12, 'LF/HF ratio': 0.45}]
{'breath_avg_len': 4.0, 'inhale_duration': 10.12, 'avg_inhale': 2.02, 'exhale_duration': 9.86, 'avg_exhale': 2.46, 'inhale_exhale_ratio': 1.03}
[{'nni_mean': 421.09, 'hr_mean': 160.29, 'hr_std': 60.09, 'rmssd': 151.25, 'lf': 232

{'breath_avg_len': 4.0, 'inhale_duration': 9.43, 'avg_inhale': 1.89, 'exhale_duration': 10.55, 'avg_exhale': 2.64, 'inhale_exhale_ratio': 0.89}
[{'nni_mean': 409.57, 'hr_mean': 164.16, 'hr_std': 58.61, 'rmssd': 139.95, 'lf': 1420.74, 'hf': 3665.72, 'LF/HF ratio': 0.39}, {'nni_mean': 453.57, 'hr_mean': 150.44, 'hr_std': 56.11, 'rmssd': 197.8, 'lf': 1066.25, 'hf': 14839.39, 'LF/HF ratio': 0.07}, {'nni_mean': 532.65, 'hr_mean': 130.47, 'hr_std': 50.01, 'rmssd': 262.4, 'lf': 22590.21, 'hf': 39486.68, 'LF/HF ratio': 0.57}, {'nni_mean': 432.5, 'hr_mean': 151.83, 'hr_std': 51.97, 'rmssd': 108.6, 'lf': 1308.15, 'hf': 3342.11, 'LF/HF ratio': 0.39}, {'nni_mean': 510.29, 'hr_mean': 131.9, 'hr_std': 46.51, 'rmssd': 258.84, 'lf': 12250.57, 'hf': 26504.66, 'LF/HF ratio': 0.46}]
{'breath_avg_len': 4.0, 'inhale_duration': 9.41, 'avg_inhale': 2.35, 'exhale_duration': 10.57, 'avg_exhale': 2.64, 'inhale_exhale_ratio': 0.89}
[{'nni_mean': 453.57, 'hr_mean': 150.44, 'hr_std': 56.11, 'rmssd': 197.8, 'lf': 1

...

[{'nni_mean': 463.66, 'hr_mean': 138.39, 'hr_std': 38.48, 'rmssd': 182.93, 'lf': 443.16, 'hf': 3677.46, 'LF/HF ratio': 0.12}, {'nni_mean': 463.66, 'hr_mean': 138.39, 'hr_std': 38.48, 'rmssd': 182.93, 'lf': 443.16, 'hf': 3677.46, 'LF/HF ratio': 0.12}, {'nni_mean': 463.9, 'hr_mean': 138.31, 'hr_std': 38.48, 'rmssd': 183.74, 'lf': 317.64, 'hf': 3987.26, 'LF/HF ratio': 0.08}, {'nni_mean': 462.14, 'hr_mean': 138.68, 'hr_std': 38.08, 'rmssd': 181.73, 'lf': 310.32, 'hf': 4159.51, 'LF/HF ratio': 0.07}, {'nni_mean': 442.33, 'hr_mean': 146.3, 'hr_std': 43.58, 'rmssd': 177.88, 'lf': 1438.43, 'hf': 1744.23, 'LF/HF ratio': 0.82}]
{'breath_avg_len': 3.33, 'inhale_duration': 11.34, 'avg_inhale': 1.89, 'exhale_duration': 8.64, 'avg_exhale': 1.73, 'inhale_exhale_ratio': 1.31}
inhale:  1.89
[{'nni_mean': 463.66, 'hr_mean': 138.39, 'hr_std': 38.48, 'rmssd': 182.93, 'lf': 443.16, 'hf': 3677.46, 'LF/HF ratio': 0.12}, {'nni_mean': 463.9, 'hr_mean': 138.31, 'hr_std': 38.48, 'rmssd': 183.74, 'lf': 317.64, 'hf

[{'nni_mean': 1418.08, 'hr_mean': 62.55, 'hr_std': 40.86, 'rmssd': 1177.04, 'lf': 559465.32, 'hf': 531105.34, 'LF/HF ratio': 1.05}, {'nni_mean': 1019.58, 'hr_mean': 83.49, 'hr_std': 45.35, 'rmssd': 1118.28, 'lf': 553331.76, 'hf': 1172251.53, 'LF/HF ratio': 0.47}, {'nni_mean': -13.23, 'hr_mean': 139.41, 'hr_std': 48.02, 'rmssd': 3094.28, 'lf': 32949.86, 'hf': 18043.84, 'LF/HF ratio': 1.83}, {'nni_mean': -11.68, 'hr_mean': 130.71, 'hr_std': 43.28, 'rmssd': 3280.54, 'lf': 55124.56, 'hf': 35042.8, 'LF/HF ratio': 1.57}, {'nni_mean': 473.17, 'hr_mean': 134.89, 'hr_std': 36.41, 'rmssd': 185.22, 'lf': 197.14, 'hf': 7717.21, 'LF/HF ratio': 0.03}]
{'breath_avg_len': 3.33, 'inhale_duration': 10.15, 'avg_inhale': 2.03, 'exhale_duration': 9.83, 'avg_exhale': 1.64, 'inhale_exhale_ratio': 1.03}
[{'nni_mean': 1019.58, 'hr_mean': 83.49, 'hr_std': 45.35, 'rmssd': 1118.28, 'lf': 553331.76, 'hf': 1172251.53, 'LF/HF ratio': 0.47}, {'nni_mean': -13.23, 'hr_mean': 139.41, 'hr_std': 48.02, 'rmssd': 3094.28, '

[{'nni_mean': 460.73, 'hr_mean': 139.44, 'hr_std': 40.85, 'rmssd': 177.94, 'lf': 162.71, 'hf': 5354.77, 'LF/HF ratio': 0.03}, {'nni_mean': 458.1, 'hr_mean': 140.47, 'hr_std': 41.29, 'rmssd': 178.84, 'lf': 137.08, 'hf': 4877.9, 'LF/HF ratio': 0.03}, {'nni_mean': 454.76, 'hr_mean': 143.19, 'hr_std': 46.78, 'rmssd': 177.99, 'lf': 131.38, 'hf': 4552.99, 'LF/HF ratio': 0.03}, {'nni_mean': -6.8, 'hr_mean': 140.91, 'hr_std': 51.41, 'rmssd': 3082.69, 'lf': 26886.07, 'hf': 21671.44, 'LF/HF ratio': 1.24}, {'nni_mean': 451.63, 'hr_mean': 144.33, 'hr_std': 46.9, 'rmssd': 182.91, 'lf': 193.09, 'hf': 3734.77, 'LF/HF ratio': 0.05}]
{'breath_avg_len': 4.0, 'inhale_duration': 9.86, 'avg_inhale': 1.97, 'exhale_duration': 10.12, 'avg_exhale': 2.02, 'inhale_exhale_ratio': 0.97}
exhale:  2.02
[{'nni_mean': 458.1, 'hr_mean': 140.47, 'hr_std': 41.29, 'rmssd': 178.84, 'lf': 137.08, 'hf': 4877.9, 'LF/HF ratio': 0.03}, {'nni_mean': 454.76, 'hr_mean': 143.19, 'hr_std': 46.78, 'rmssd': 177.99, 'lf': 131.38, 'hf':

[{'nni_mean': 544.55, 'hr_mean': 125.53, 'hr_std': 37.65, 'rmssd': 406.71, 'lf': 21659.07, 'hf': 26742.97, 'LF/HF ratio': 0.81}, {'nni_mean': 463.17, 'hr_mean': 142.61, 'hr_std': 47.99, 'rmssd': 163.02, 'lf': 1991.76, 'hf': 2125.19, 'LF/HF ratio': 0.94}, {'nni_mean': -12.07, 'hr_mean': 145.38, 'hr_std': 57.87, 'rmssd': 3073.35, 'lf': 18260.4, 'hf': 15407.31, 'LF/HF ratio': 1.19}, {'nni_mean': 562.58, 'hr_mean': 121.95, 'hr_std': 40.2, 'rmssd': 350.7, 'lf': 6431.08, 'hf': 22720.18, 'LF/HF ratio': 0.28}, {'nni_mean': 488.92, 'hr_mean': 132.45, 'hr_std': 40.12, 'rmssd': 198.21, 'lf': 1278.95, 'hf': 2628.34, 'LF/HF ratio': 0.49}]
{'breath_avg_len': 4.0, 'inhale_duration': 9.74, 'avg_inhale': 1.95, 'exhale_duration': 10.24, 'avg_exhale': 2.05, 'inhale_exhale_ratio': 0.95}
[{'nni_mean': 463.17, 'hr_mean': 142.61, 'hr_std': 47.99, 'rmssd': 163.02, 'lf': 1991.76, 'hf': 2125.19, 'LF/HF ratio': 0.94}, {'nni_mean': -12.07, 'hr_mean': 145.38, 'hr_std': 57.87, 'rmssd': 3073.35, 'lf': 18260.4, 'hf':

[{'nni_mean': 443.49, 'hr_mean': 156.83, 'hr_std': 58.98, 'rmssd': 213.36, 'lf': 13298.24, 'hf': 7478.82, 'LF/HF ratio': 1.78}, {'nni_mean': 547.06, 'hr_mean': 135.94, 'hr_std': 58.35, 'rmssd': 458.86, 'lf': 18598.69, 'hf': 18808.22, 'LF/HF ratio': 0.99}, {'nni_mean': 547.06, 'hr_mean': 135.94, 'hr_std': 58.35, 'rmssd': 458.86, 'lf': 18598.69, 'hf': 18808.22, 'LF/HF ratio': 0.99}, {'nni_mean': 441.82, 'hr_mean': 157.81, 'hr_std': 59.34, 'rmssd': 251.92, 'lf': 15529.88, 'hf': 16248.07, 'LF/HF ratio': 0.96}, {'nni_mean': 440.7, 'hr_mean': 158.64, 'hr_std': 59.79, 'rmssd': 255.19, 'lf': 15782.45, 'hf': 23222.08, 'LF/HF ratio': 0.68}]
{'breath_avg_len': 4.0, 'inhale_duration': 9.75, 'avg_inhale': 2.44, 'exhale_duration': 10.23, 'avg_exhale': 2.05, 'inhale_exhale_ratio': 0.95}
[{'nni_mean': 547.06, 'hr_mean': 135.94, 'hr_std': 58.35, 'rmssd': 458.86, 'lf': 18598.69, 'hf': 18808.22, 'LF/HF ratio': 0.99}, {'nni_mean': 547.06, 'hr_mean': 135.94, 'hr_std': 58.35, 'rmssd': 458.86, 'lf': 18598.69

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.plot(resp_data)

In [ ]:
HRV.current_trends

In [ ]:
B.features['avg_inhale']

In [2]:
actuator_ip = '192.168.0.100'
actuator_port = 12000

osc_client = SimpleUDPClient(actuator_ip, actuator_port) 
osc_client.send_message("/actuator/inflate", 0.0)
